In [25]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
df = pd.read_csv('df1_loan.csv', index_col =0)


del df["Loan_ID"]

df["Total_Income"]=df["Total_Income"].str.replace("$","")
df["Total_Income"]=df["Total_Income"].apply(pd.to_numeric)

df["Dependents"]=df["Dependents"].str.replace('+','')
df["Dependents"]=df["Dependents"].apply(pd.to_numeric)


df = df.dropna()

def get_features_target():

    target_column = f"Loan_Status"
    target = df[target_column].dropna()

    df2 = df.copy()
    del df2[target_column]

    return df2, target

features, target = get_features_target()

features["Self_Employed"] = features["Self_Employed"].astype('category').cat.codes
features["Gender"] = features["Gender"].astype('category').cat.codes
features["Married"] = features["Married"].astype('category').cat.codes
features["Education"] = features["Education"].astype('category').cat.codes
features["Property_Area"] = features["Property_Area"].astype('category').cat.codes


features


# def k_best_features(k, features, target):
    
#     X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=3000)
  
#     selector = SelectKBest(f_regression, k = k)
#     selector.fit(X_train, y_train)

#     selected = selector.get_support()
  
#     columns = []
#     for i in range(len(selected)):
#         if selected[i]:
#             columns.append(df.columns[i])

#     return columns



,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Total_Income
1,1,1,1.0,0,0,4583,1508.0,128.0,360.0,1.0,0,6091.0
2,1,1,0.0,0,1,3000,0.0,66.0,360.0,1.0,2,3000.0
3,1,1,0.0,1,0,2583,2358.0,120.0,360.0,1.0,2,4941.0
4,1,0,0.0,0,0,6000,0.0,141.0,360.0,1.0,2,6000.0
5,1,1,2.0,0,1,5417,4196.0,267.0,360.0,1.0,2,9613.0
...,...,...,...,...,...,...,...,...,...,...,...,...
494,1,1,0.0,0,0,3597,2157.0,119.0,360.0,0.0,0,5754.0
495,0,1,1.0,0,0,3326,913.0,105.0,84.0,1.0,1,4239.0
496,1,1,0.0,1,0,2600,1700.0,107.0,360.0,1.0,0,4300.0
498,1,1,1.0,0,1,2895,0.0,95.0,360.0,1.0,1,2895.0


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
data_rows = []



for i in range(15):
    X_train, X_test, y_train, y_test = train_test_split(features, target)
    estimator = LinearSVC()
    estimator.fit(X_train, y_train)
    data_rows.append({
        "score": estimator.score(X_test, y_test)
    })

test_df = pd.DataFrame(data_rows)
test_df[::5]

C:\Users\maxnb\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\maxnb\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\maxnb\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\maxnb\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\maxnb\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

,score
0,0.561224
5,0.755102
10,0.357143


In [27]:
import numpy as np 
def plot_NN_data(x_train, y_train):
    x_t0, x_t1 = x_train[np.where(y_train == 0)[0]], x_train[np.where(y_train == 1)[0]]
    plt.scatter(x_t0[:,0], x_t0[:,1], color='blue', label='class c:-1', marker='s')
    plt.scatter(x_t1[:,0], x_t1[:,1], color='red', label='class c:1', marker="o")

    plt.legend()
    plt.xlabel('x1')
    plt.ylabel('x2')
    ax = plt.gca()
    ax.autoscale(False)
    plt.show()

#sns.pairplot(df, hue='Loan_Status')
#f = features["ApplicantIncome"].zip(features["Gender"])
#f
#plot_NN_data(f, target)